In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [2]:
df = pd.read_csv("edu_enrollees.csv")
df.head(2)

enrollee_id      city  city_development_index gender   
0       8949.0  city_103                   0.920   Male  \
1      29725.0   city_40                   0.776   Male   

       relevant_experience enrolled_university education_level   
0  Has relevant experience       no_enrollment        Graduate  \
1   No relevant experience       no_enrollment        Graduate   

  major_discipline experience company_size company_type last_new_job   
0             STEM        >20          NaN          NaN            1  \
1             STEM         15        50-99      Pvt Ltd           >4   

   training_hours  target   Xgrp  
0            36.0     1.0  train  
1            47.0     0.0  train

### 전처리

In [3]:
df = df.drop(columns = ["city", "company_size", "company_type"])
df.head(2)

enrollee_id  city_development_index gender      relevant_experience   
0       8949.0                   0.920   Male  Has relevant experience  \
1      29725.0                   0.776   Male   No relevant experience   

  enrolled_university education_level major_discipline experience   
0       no_enrollment        Graduate             STEM        >20  \
1       no_enrollment        Graduate             STEM         15   

  last_new_job  training_hours  target   Xgrp  
0            1            36.0     1.0  train  
1           >4            47.0     0.0  train

In [4]:
df.dtypes

enrollee_id               float64
city_development_index    float64
gender                     object
relevant_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
last_new_job               object
training_hours            float64
target                    float64
Xgrp                       object
dtype: object

In [5]:
df.isna().sum()

enrollee_id                  0
city_development_index       0
gender                    4508
relevant_experience          0
enrolled_university        386
education_level            460
major_discipline          2813
experience                  65
last_new_job               423
training_hours               0
target                       0
Xgrp                         0
dtype: int64

In [6]:
df = df.dropna()
len(df)

12477

In [7]:
df.isna().sum().sum()

0

In [8]:
df["experience"].unique(), df["last_new_job"].unique()

(array(['>20', '15', '13', '7', '5', '16', '4', '11', '<1', '18', '19',
        '12', '10', '9', '2', '6', '14', '3', '8', '20', '17', '1'],
       dtype=object),
 array(['1', '>4', '4', '3', '2', 'never'], dtype=object))

In [9]:
df = df.loc[~df["experience"].isin([">20", "<1"]), ]
df = df.loc[~df["last_new_job"].isin([">4", "never"]), ]
df["experience"] = df["experience"].astype("int")
df["last_new_job"] = df["last_new_job"].astype("int")

In [10]:
len(df)

7522

In [11]:
df_base = df.reset_index(drop = True)
df_base.head()

enrollee_id  city_development_index  gender      relevant_experience   
0      27107.0                   0.920    Male  Has relevant experience  \
1      23853.0                   0.920    Male  Has relevant experience   
2      11399.0                   0.827  Female  Has relevant experience   
3      31972.0                   0.843    Male  Has relevant experience   
4      19061.0                   0.926    Male  Has relevant experience   

  enrolled_university education_level major_discipline  experience   
0       no_enrollment        Graduate             STEM           7  \
1       no_enrollment        Graduate             STEM           5   
2       no_enrollment        Graduate             Arts           4   
3       no_enrollment         Masters             STEM          11   
4       no_enrollment         Masters             STEM          11   

   last_new_job  training_hours  target   Xgrp  
0             1            46.0     1.0  train  
1             1           108.0     0.0  train  
2             1           132.0     1.0  train  
3             1            68.0     0.0  train  
4             2            50.0     0.0  train

### Q1.

In [13]:
df_q1 = df_base[["relevant_experience", "target"]].copy()
df_q1.head(2)

relevant_experience  target
0  Has relevant experience     1.0
1  Has relevant experience     0.0

In [14]:
df_q1["relevant_experience"].unique()

array(['Has relevant experience', 'No relevant experience'], dtype=object)

In [15]:
df_q1_A = df_q1.loc[df_q1["relevant_experience"] == "No relevant experience", ]
df_q1_B = df_q1.loc[df_q1["relevant_experience"] == "Has relevant experience", ]

In [19]:
# df_q1_A["target"].value_counts()
stat_A = df_q1_A["target"].value_counts(normalize = True)[1]
stat_A

0.38287331917905165

In [20]:
stat_B = df_q1_B["target"].value_counts(normalize = True)[1]
stat_B

0.21591095105581928

In [22]:
pd.crosstab(df_q1["relevant_experience"], df_q1["target"],
            normalize = "index")

target                        0.0       1.0
relevant_experience                        
Has relevant experience  0.784089  0.215911
No relevant experience   0.617127  0.382873

In [23]:
round(stat_A / stat_B, 2)

1.77

In [24]:
df_q1.groupby("relevant_experience")["target"].mean()

relevant_experience
Has relevant experience    0.215911
No relevant experience     0.382873
Name: target, dtype: float64

### Q2.

In [26]:
df_q2_cat = df_base.loc[:, "gender":"major_discipline"]
df_q2_cat.head(1)

gender      relevant_experience enrolled_university education_level   
0   Male  Has relevant experience       no_enrollment        Graduate  \

  major_discipline  
0             STEM

In [28]:
# df_q2_dum = pd.get_dummies(df_q2_cat, columns = df_q2_cat.columns) # 시험버전
df_q2_dum = pd.get_dummies(df_q2_cat, dtype = "int")
df_q2_dum.head(2)

gender_Female  gender_Male  gender_Other   
0              0            1             0  \
1              0            1             0   

   relevant_experience_Has relevant experience   
0                                            1  \
1                                            1   

   relevant_experience_No relevant experience   
0                                           0  \
1                                           0   

   enrolled_university_Full time course  enrolled_university_Part time course   
0                                     0                                     0  \
1                                     0                                     0   

   enrolled_university_no_enrollment  education_level_Graduate   
0                                  1                         1  \
1                                  1                         1   

   education_level_Masters  education_level_Phd  major_discipline_Arts   
0                        0                    0                      0  \
1                        0                    0                      0   

   major_discipline_Business Degree  major_discipline_Humanities   
0                                 0                            0  \
1                                 0                            0   

   major_discipline_No Major  major_discipline_Other  major_discipline_STEM  
0                          0                       0                      1  
1                          0                       0                      1

In [ ]:
df_q2_dum.columns.to_series().reset_index()

In [34]:
df_q2_dum.columns[[2, 4, 7, 10, 16]]

Index(['gender_Other', 'relevant_experience_No relevant experience',
       'enrolled_university_no_enrollment', 'education_level_Phd',
       'major_discipline_STEM'],
      dtype='object')

In [35]:
df_q2_dum = df_q2_dum.drop(columns = df_q2_dum.columns[[2, 4, 7, 10, 16]], )

In [36]:
df_q2_dum.head(2)

gender_Female  gender_Male  relevant_experience_Has relevant experience   
0              0            1                                            1  \
1              0            1                                            1   

   enrolled_university_Full time course  enrolled_university_Part time course   
0                                     0                                     0  \
1                                     0                                     0   

   education_level_Graduate  education_level_Masters  major_discipline_Arts   
0                         1                        0                      0  \
1                         1                        0                      0   

   major_discipline_Business Degree  major_discipline_Humanities   
0                                 0                            0  \
1                                 0                            0   

   major_discipline_No Major  major_discipline_Other  
0                          0                       0  
1                          0                       0

In [25]:
df_base.columns

Index(['enrollee_id', 'city_development_index', 'gender',
       'relevant_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'last_new_job', 'training_hours',
       'target', 'Xgrp'],
      dtype='object')

In [37]:
col1 = ["target", "Xgrp"]
col2 = ["city_development_index", "experience", "last_new_job", "training_hours"]

df_job2 = pd.concat([df_base[col1 + col2],
                     df_q2_dum],
                    axis = 1)
df_job2.head(2)

target   Xgrp  city_development_index  experience  last_new_job   
0     1.0  train                    0.92           7             1  \
1     0.0  train                    0.92           5             1   

   training_hours  gender_Female  gender_Male   
0            46.0              0            1  \
1           108.0              0            1   

   relevant_experience_Has relevant experience   
0                                            1  \
1                                            1   

   enrolled_university_Full time course  enrolled_university_Part time course   
0                                     0                                     0  \
1                                     0                                     0   

   education_level_Graduate  education_level_Masters  major_discipline_Arts   
0                         1                        0                      0  \
1                         1                        0                      0   

   major_discipline_Business Degree  major_discipline_Humanities   
0                                 0                            0  \
1                                 0                            0   

   major_discipline_No Major  major_discipline_Other  
0                          0                       0  
1                          0                       0

In [ ]:
model_lr = LogisticRegression(C = 100000, max_iter = 1000,
                              solver = "liblinear",
                              random_state = 123)
model_lr.fit(X = df_job2.drop(columns = ["target", "Xgrp"]),
             y = df_job2["target"])

In [41]:
np.exp(model_lr.coef_) # OR(Odds Ratio)

array([[0.00220435, 0.97190078, 1.10037739, 0.99907422, 0.84961169,
        0.87221096, 0.46740116, 1.67214935, 0.75465354, 1.38142616,
        1.01688281, 1.29909128, 1.11361023, 1.27567972, 1.48869549,
        0.64443085]])

In [54]:
np.exp(model_lr.coef_).max()

1.6721493496611732

In [42]:
pd.DataFrame(np.exp(model_lr.coef_), 
             columns = df_job2.columns[2:])

city_development_index  experience  last_new_job  training_hours   
0                0.002204    0.971901      1.100377        0.999074  \

   gender_Female  gender_Male  relevant_experience_Has relevant experience   
0       0.849612     0.872211                                     0.467401  \

   enrolled_university_Full time course  enrolled_university_Part time course   
0                              1.672149                              0.754654  \

   education_level_Graduate  education_level_Masters  major_discipline_Arts   
0                  1.381426                 1.016883               1.299091  \

   major_discipline_Business Degree  major_discipline_Humanities   
0                           1.11361                      1.27568  \

   major_discipline_No Major  major_discipline_Other  
0                   1.488695                0.644431

### Q3.

In [47]:
df_train = df_job2.loc[df_job2["Xgrp"] == "train", ] 
df_test  = df_job2.loc[df_job2["Xgrp"] == "test",  ] 

In [50]:
df_train.shape

(4706, 18)

In [51]:
model_knn = KNeighborsClassifier(n_neighbors = 5)
model_knn.fit(X = df_train.drop(columns = ["Xgrp", "target"]),
              y = df_train["target"])

In [52]:
pred = model_knn.predict(df_test.drop(columns = ["Xgrp", "target"]))
pred[:4]

array([0., 0., 0., 0.])

In [53]:
pd.crosstab(df_test["target"], pred)

col_0    0.0  1.0
target           
0.0     1899  193
1.0      616  108

In [56]:
pd.crosstab(df_test["target"], pred).values.diagonal()

array([1899,  108], dtype=int64)

In [57]:
from sklearn.metrics import accuracy_score

In [58]:
round(accuracy_score(y_true = df_test["target"], 
                     y_pred = pred), 2)

0.71